[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/daradija/pyroboadvisor/blob/main/colab_b.ipynb)

# Agent IA - Py Robo Advisor

# Primero lo primero... A1 vs S&P500

Antes de poder replicar A1 en la BMV necesitamos correr A1 normal (en aras de tener un entendimiento claro del paso a paso).
Dado esta en esta sección se correrá A1

In [ ]:
!apt-get install -y -qq git
!git clone https://github.com/daradija/pyroboadvisor.git
%cd pyroboadvisor
!pip install -r requirements.txt

Aquí se tendría que correr

Para correr el pipiline de A1, dado que en este documento nos interesa entrar al detalle lo que haremos será pegar el código de sample.py que se encuentra en el repositorio, lo partiremos en distintos chunks para entender (y cuidar la ejecución ya que en findes el servidor de A1 tiene limitaciones por licencia para solo correr 1 vez por día la estrategia)

In [ ]:
# Sample.py - Parte 1
from pyroboadvisor.pyroboadvisor import PyRoboAdvisor
import pandas as pd

# partir en servicio web
# usuario y registrar uso
# El simulador se ejecuta en los dos sitios y manda hash.
today = pd.Timestamp.now().normalize()
stoday = today.strftime("%Y-%m-%d")
p={
    "fecha_inicio": "2019-01-01",
    "fecha_fin": stoday,
    "money": 100000,
    "numberStocksInPortfolio": 10,
    "orderMarginBuy": 0.005,  # margen de ordenes de compra y venta
    "orderMarginSell": 0.005,  # margen de ordenes de compra y venta
    "apalancamiento": 10 / 6,  # apalancamiento de las compras
    "ring_size": 252,
    "rlog_size": 22,
    "cabeza": 5,
    "seeds": 100,
    "percentil": 95,
    "prediccion": 5,
    "multiploMantenimiento": 6,
    "b":True,

    "key": "",
    "email": "",
}



p["rlog_size"] = [11,22,44]
from pyroboadvisor.download_us_money_supply import MakerUsMoneySupply
mum = MakerUsMoneySupply("2018-01-01")
usms=[]
for meses in [6,12]:
    for percentil in [10,20,30,40,50,60,70,80,90]:
        usms.append(mum.get(meses,percentil).date2usms)
p["signoMultiplexado"]=list(range(len(usms)))


pra=PyRoboAdvisor(p)


pra.readTickersFromWikipedia()
pra.completeTickersWithIB()  # Completa los tickers de IB que no están en el SP500, para que pueda venderlos

pra.prepare(signoMultiplexado=usms)  # Prepara los datos y la estrategia
pra.simulate()

pra.automatizeOrders()
